# **LearningManager**: Create a Federated Learning Pipeline using MedFL
<img src='../Images/MEDfl_Diagramm.png' />

The `Learning Manager`, serving as the second subpackage of *Medfl*, extends the work initiated by the `NetManager`. It operates by taking the federated dataset as its input and, through its processes, generates a **Trained model**. Additionally, it **stores relevant Results** within the database.

<b>The Federated module</b>  of <b>MEDfl</b>, which is based on the <a href="https://flower.dev/">Flower</a> framework, is well designed for doing this task, it's mainly follows the flower paradigm, but with many customization, to deal with the SQL database, and to be homogeneous with the <b>NetManager</b>


<img src='../Images/LearningManager.png' width="50%" />


In [1]:
from Medfl.LearningManager.utils import global_params

import sys
sys.path.append(global_params['base_url'])

import os
os.environ['PYTHONPATH'] = global_params['base_url']

/bin/python3


Imports

In [2]:
# Torch imports
import torch
import torch.nn as nn
import torch.optim as optim

# flower imports 
import flwr as fl

# Medfl imports 
from Medfl.LearningManager.dynamicModal import DynamicModel
from Medfl.LearningManager.model import Model
from Medfl.LearningManager.strategy import Strategy
from Medfl.LearningManager.server import *

from Medfl.NetManager.flsetup import FLsetup
from Medfl.LearningManager.flpipeline import *

from Medfl.LearningManager.plot import AccuracyLossPlotter




2024-02-13 15:55:46.500132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1. Modal inialization

The first step involves initializing the model to subsequently transmit it to the central server. MedFL offers two distinct methods for model initialization:

1. **Without Utilizing Transfer Learning:** This method is employed in scenarios where a pretrained model is unavailable. Here, the initial model is constructed from scratch, and subsequently transmitted to the central server (Simple Federated Learning).

2. **Utilizing Transfer Learning:** This approach comes into play when a pretrained model exists, and the objective is to fine-tune it within the federated learning network (Federated-Transfer Learning).

<img src='../Images/Modelinit.png' width="80%" style="display: block  ; margin : 0 auto" />

### 1.1 without activation of Transfer Learning

in this methode we can eather specify the structure of our neural network ( number and size of layers, activation function ) or we can write define our modal and write the code from scratch and pass it to the server 


As MEDfl is based on flower, it has only 3 models based on torch neural nets  :
<ul>
    <li> Binary Classifier </li>
    <li> Multiclass Classifier</li>
    <li> Linear Regressor</li>
</ul>

the choice of the model and all machine learning parameters (learning rate, eopch size, train epochs), should be declared on a YAML file. `Medfl/learningManager/params.yaml`


A. Create the modal from scratch 

In [3]:
class BinaryClassifier(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):

    super(BinaryClassifier, self).__init__()
    self.fc1 = nn.Linear(input_dim,hidden_dim)
    self.fc2 = nn.Linear(hidden_dim,2*hidden_dim)
    self.fc3 = nn.Linear(2*hidden_dim,output_dim)

  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))#softmax
    return x

# Create an instance of BinaryClassifier 
# The input dim is the number of columns in the federated DataSet 
binary_classifier= BinaryClassifier(17,6,1)
# define the optimizer and criterion
optimizer,criterion = optim.SGD(binary_classifier.parameters(), lr=0.0001),nn.BCELoss()

# Creating a new Model instance using the specific model created by DynamicModel
global_model = Model(binary_classifier, optimizer, criterion)

# The modal to be passed to the server 
global_model


B. Create a Dynamic Using the DynamicModal class

In [4]:
# Creating a DynamicModel instance
dynamic_model = DynamicModel()

# Creating a specific model dynamically using DynamicModel
specific_model = dynamic_model.create_model(model_type='Logistic Regressor', params_dict={"input_dim": 17, "output_dim": 1})

# Pass the specific_model to the Model class
optimizer = optim.SGD(specific_model.parameters(), lr=0.0001)
criterion = nn.BCELoss()


# Creating a new Model instance using the specific model created by DynamicModel
global_model = Model(specific_model, optimizer, criterion)

# The modal to be passed to the server
global_model.get_parameters()

[array([[-0.1753857 ,  0.01182628, -0.0656938 , -0.23282903,  0.20984195,
          0.10602364,  0.18930514,  0.20693147,  0.04460086,  0.09694176,
          0.11027294,  0.1468007 , -0.23502725, -0.18373561,  0.16646941,
         -0.1295526 ,  0.09775756]], dtype=float32),
 array([0.05561392], dtype=float32)]

### 1.2 Using the Transfer Learning 

in this method we will use a pretrained modal that we showed on the 3rd tutorial `3_transferLearning-checkpoint` 

In [5]:
# Define the neural network model using PyTorch
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.3)
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x

In [6]:
# Load the pretrained model
model = Model.load_model("binary_classifier")


# Pass the specific_model to the Model class
optimizer = optim.SGD(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

# Creating a new Model instance using the specific model created by DynamicModel
global_model = Model(model, optimizer, criterion)

# Get the initial params of the model 
init_params = global_model.get_parameters() ; 

print(model)

BinaryClassifier(
  (fc1): Linear(in_features=17, out_features=64, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


### 2. Define the Aggregation Algorithms (**Aggregation Strategy**)

Aggregation algorithms are the mechanizms used by the server to produce a joint unified model, they are known as <b>Strategies</b> on <b>Flower</b>,  in the literature we have many aggregation algorithms, it's true that not all of them are implemented in flower, but we have the most used ones:
 <ul>
    <li><a href = "arxiv.org/abs/1602.05629">Federated Averaging (FedAvg)</a></li>
    <li><a href = "arxiv.org/abs/2003.00295">Adaptive Federated Optimization using Adam (FedAdam)</a></li>
    <li><a href = "arxiv.org/pdf/1803.01498v1.pdf">Federated Median (FedMedian)</a></li>
    <li><a href = "arxiv.org/pdf/1909.06335.pdf">Federated Averaging with Momentum (FedAvgM)</a></li>
    <li><a href = "arxiv.org/abs/2003.00295">Adaptive Federated Optimization using Yogi (FedYogi)</a></li>
    <li><a href = "arxiv.org/abs/2003.00295">Adaptive Federated Optimization (FedOpt)</a></li>
    <li><a href = "arxiv.org/abs/1812.06127">Federated Optimization (FedProx)</a></li>
    <li><a href = "openreview.net/pdf?id=ByexElSYDr">FAIR RESOURCE ALLOCATION IN FEDERATED LEARNING (QFedAvg)</a></li>
    <li><a href = "arxiv.org/abs/2003.00295">Adaptive Federated Optimization using Adagrad (FedAdagrad)</a></li>
 </ul>
We can also find them on the strategy directory on Flower.

In [7]:
# List all available aggregation strategies on flower 
dir(fl.server.strategy)

['Bulyan',
 'DPFedAvgAdaptive',
 'DPFedAvgFixed',
 'FaultTolerantFedAvg',
 'FedAdagrad',
 'FedAdam',
 'FedAvg',
 'FedAvgAndroid',
 'FedAvgM',
 'FedMedian',
 'FedOpt',
 'FedProx',
 'FedTrimmedAvg',
 'FedXgbBagging',
 'FedXgbCyclic',
 'FedXgbNnAvg',
 'FedYogi',
 'Krum',
 'QFedAvg',
 'Strategy',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'aggregate',
 'bulyan',
 'dpfedavg_adaptive',
 'dpfedavg_fixed',
 'fault_tolerant_fedavg',
 'fedadagrad',
 'fedadam',
 'fedavg',
 'fedavg_android',
 'fedavgm',
 'fedmedian',
 'fedopt',
 'fedprox',
 'fedtrimmedavg',
 'fedxgb_bagging',
 'fedxgb_cyclic',
 'fedxgb_nn_avg',
 'fedyogi',
 'krum',
 'qfedavg',
 'strategy']

<div style="display : flex"> 
<div >The creation of a strategy is based on the "Strategy" Class, it's enough to pass the name of one of the mentionned strategies, and other federated learning parameters which are used during federation rounds.

<ul>
    <li><u>fraction_fit :</u> the fraction of clients to sample in order  to train the model </li>
    <li><u>fraction_evaluate  :</u> the fraction of clients to sample in order to evaluate the model (validation) </li>
    <li><u>min_fit_clients :</u>  the minimum number of clients to sample for the training on each round</li>
    <li><u>min_evaluate_clients :</u> the minimum number of clients to be sampled for the evaluation on each round </li>
    <li><u>min_available_clients :</u> the minimum  required number of available clients to start a federation round </li>
<ul></div>
<div>
<img src='../Images/strategy.png' width="80%" style="display: block  ; margin : 0 auto" />
</div>
</div>



In [8]:
# Create the strategy
fed_avg = Strategy("FedAvg", 
                   fraction_fit = 1.0 ,
                   fraction_evaluate = 1.0,
                   min_fit_clients = 2,
                   min_evaluate_clients = 2,
                   min_available_clients = 2 , 
                   initial_parameters=global_model.get_parameters())

### 3. Federated Learning Server
The server in the FL terminology, is the set of the mechanizms that orchestrate  all the FL processes, (distributing the model to  clients, getting the models updates from clients, aggregate these updates. 

**Remark :** The FL Server has no relation ! with webservers or any other server concepts.

#### MEDfl server

<div style="display : flex"> 
<div >

The FL server on MEDfl, is based on the server class of Flower, but with our customization, because we have addeda gradient perturbing feature using [Opacus](https://opacus.ai), so we can make the FLpipeline **differentially privacy** </b> 

The FLserver in MEDfl, has the follwoing attributes : 

<ul>
    <li> <u>global_model :</u> The global model
    <li> <u>strategy :</u> The  aggregation strategy
    <li> <u>num_rounds :</u> The number of federation rounds
    <li> <u>num_clients :</u> The number of train nodes
    <li> <u>dataset :</u> The federated dataset generated by an FLsetup
    <li> <u>diff_privacy :</u> a boolean variable to  enable (disable) the Differentially Private training mode
</ul>

</div>
<div>
<img src='../Images/createServer.png' width="250" style="display: block  ; margin : 0 auto" />
</div>
</div>



For the federated Dataset will use the one generated on the 2nd tuto `2_NetManager-checkpoint`

In [9]:
# use the auto setup created on the second tuto
fl_setup = FLsetup.read_setup(1)
fl_setup.id

/home/local/USHERBROOKE/saho6810/.local/lib/python3.8/site-packages/pandas/io/sql.py:1761: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  return insp.has_table(name, schema or self.meta.schema)


1

In [10]:
# The federated dataset is already created 
fl_setup.get_flDataSet()

FedId  FLsetupId  FLpipeId                  name
0      1          1         1  Flsetup_1_Feddataset

In [11]:
# Create the federated Data set ( in our case the federated Dataset is created it will use it directly)
fl_dataset = fl_setup.create_federated_dataset(
   output="event_death", 
       fit_encode=["site_hospital", "site_region"], 
    to_drop=[ "event_death" , "id" ], 
  )

fl_dataset.size

17

In [12]:
# Create The server 
server = FlowerServer(global_model, strategy = fed_avg, num_rounds = 10,
                       num_clients  = len(fl_dataset.trainloaders), 
                       fed_dataset = fl_dataset,diff_privacy = False ,
                       # You can change the resources alocated for each client based on your machine 
                       client_resources={'num_cpus': 1.0, 'num_gpus': 0.0}
                       )

### 4. FL Pipeline Creation

<div style="display : flex"> 
<div >

The Federated pipeline is the class responsible of running the federated learning process, launch the training of the model then test it and store the results in the database 

To create a pipeline we need to provide the `name`, `description` and finaly the `server` 

</div>
<div>
<img src='../Images/pipeline.png' width="250" style="display: block  ; margin : 0 auto" />
</div>
</div>



In [13]:
# Create the pipeline 
ppl_1 = FLpipeline( name ="the first fl_pipeline",description = "this is our second FL pipeline",
                   server = server)

In [14]:
# Display the Tests nodes on the federated dataset
ppl_1.server.fed_dataset.test_nodes

['Midwest', 'South', 'West', 'Northeast']

In [15]:
print(len(ppl_1.server.fed_dataset.testloaders[0]))
print(len(ppl_1.server.fed_dataset.testloaders[1]))
print(len(ppl_1.server.fed_dataset.testloaders[2]))

18
32
93


In [16]:
# Run the Traning of the model
history = ppl_1.server.run()

INFO flwr 2024-02-13 15:55:49,410 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
02/13/2024 15:55:49:INFO:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-02-13 15:55:51,566	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-02-13 15:55:54,055 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 47203137946.0, 'GPU': 1.0, 'accelerator_type:G': 1.0, 'CPU': 24.0, 'node:__internal_head__': 1.0, 'object_store_memory': 78643200.0, 'node:10.44.86.57': 1.0}
02/13/2024 15:55:54:INFO:Flower VCE: Ray initialized with resources: {'memory': 47203137946.0, 'GPU': 1.0, 'accelerator_type:G': 1.0, 'CPU': 24.0, 'node:__internal_head__': 1.0, 'object_store_memory': 78643200.0, 'node:10.44.86.57': 1.0}
INFO flwr 2024-02-13 15:55:54,056 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
02/13/2024 15:55:54:INFO:Opt

Server-side evaluation loss 0.06630590558052063 / accuracy 0.625


(DefaultActor pid=29969) /home/local/USHERBROOKE/saho6810/.local/lib/python3.8/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
(DefaultActor pid=29969)   warnings.warn(
(DefaultActor pid=29969) /home/local/USHERBROOKE/saho6810/.local/lib/python3.8/site-packages/flwr/simulation/ray_transport/ray_actor.py:72: DeprecationWarning:  Ensure your client is of type `flwr.client.Client`. Please convert it using the `.to_client()` method before returning it in the `client_fn` you pass to `start_simulation`. We have applied this conversion on your behalf. Not returning a `Client` might trigger an error in future versions of Flower.
(DefaultActor pid=29969)   client = check_clientfn_returns_client(client_fn(cid))
DEBUG flwr 2024-02-13 15:56:01,722 | server.py:236 | fit_rou

(DefaultActor pid=29969) [Client 1] fit, config: {}
(DefaultActor pid=29969) epsilon of client 1 : eps = 0
Server-side evaluation loss 0.06550507992506027 / accuracy 0.625
Server-side evaluation loss 0.06483973562717438 / accuracy 0.625
(DefaultActor pid=29969) [Client 0] evaluate, config: {}
(DefaultActor pid=29969) [Client 1] evaluate, config: {}
Server-side evaluation loss 0.06413096934556961 / accuracy 0.625


DEBUG flwr 2024-02-13 15:56:01,937 | server.py:236 | fit_round 4 received 2 results and 0 failures
02/13/2024 15:56:01:DEBUG:fit_round 4 received 2 results and 0 failures
INFO flwr 2024-02-13 15:56:01,945 | server.py:125 | fit progress: (4, 0.06348368525505066, {'accuracy': 0.625}, 7.84773136900003)
02/13/2024 15:56:01:INFO:fit progress: (4, 0.06348368525505066, {'accuracy': 0.625}, 7.84773136900003)
DEBUG flwr 2024-02-13 15:56:01,946 | server.py:173 | evaluate_round 4: strategy sampled 2 clients (out of 2)
02/13/2024 15:56:01:DEBUG:evaluate_round 4: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-02-13 15:56:01,969 | server.py:187 | evaluate_round 4 received 2 results and 0 failures
02/13/2024 15:56:01:DEBUG:evaluate_round 4 received 2 results and 0 failures
DEBUG flwr 2024-02-13 15:56:01,970 | server.py:222 | fit_round 5: strategy sampled 2 clients (out of 2)
02/13/2024 15:56:01:DEBUG:fit_round 5: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-02-13 15:56:02,007 | server

Server-side evaluation loss 0.06348368525505066 / accuracy 0.625
Server-side evaluation loss 0.06283245980739594 / accuracy 0.625
Server-side evaluation loss 0.06223991513252258 / accuracy 0.625


DEBUG flwr 2024-02-13 15:56:02,168 | server.py:236 | fit_round 7 received 2 results and 0 failures
02/13/2024 15:56:02:DEBUG:fit_round 7 received 2 results and 0 failures
INFO flwr 2024-02-13 15:56:02,178 | server.py:125 | fit progress: (7, 0.061645135283470154, {'accuracy': 0.625}, 8.081435993999548)
02/13/2024 15:56:02:INFO:fit progress: (7, 0.061645135283470154, {'accuracy': 0.625}, 8.081435993999548)
DEBUG flwr 2024-02-13 15:56:02,180 | server.py:173 | evaluate_round 7: strategy sampled 2 clients (out of 2)
02/13/2024 15:56:02:DEBUG:evaluate_round 7: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-02-13 15:56:02,210 | server.py:187 | evaluate_round 7 received 2 results and 0 failures
02/13/2024 15:56:02:DEBUG:evaluate_round 7 received 2 results and 0 failures
DEBUG flwr 2024-02-13 15:56:02,211 | server.py:222 | fit_round 8: strategy sampled 2 clients (out of 2)
02/13/2024 15:56:02:DEBUG:fit_round 8: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-02-13 15:56:02,251 | se

Server-side evaluation loss 0.061645135283470154 / accuracy 0.625
Server-side evaluation loss 0.06106271594762802 / accuracy 0.625
Server-side evaluation loss 0.06058136373758316 / accuracy 0.625


DEBUG flwr 2024-02-13 15:56:02,407 | server.py:236 | fit_round 10 received 2 results and 0 failures
02/13/2024 15:56:02:DEBUG:fit_round 10 received 2 results and 0 failures
INFO flwr 2024-02-13 15:56:02,414 | server.py:125 | fit progress: (10, 0.06007593870162964, {'accuracy': 0.625}, 8.316932846999407)
02/13/2024 15:56:02:INFO:fit progress: (10, 0.06007593870162964, {'accuracy': 0.625}, 8.316932846999407)
DEBUG flwr 2024-02-13 15:56:02,415 | server.py:173 | evaluate_round 10: strategy sampled 2 clients (out of 2)
02/13/2024 15:56:02:DEBUG:evaluate_round 10: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-02-13 15:56:02,439 | server.py:187 | evaluate_round 10 received 2 results and 0 failures
02/13/2024 15:56:02:DEBUG:evaluate_round 10 received 2 results and 0 failures
INFO flwr 2024-02-13 15:56:02,440 | server.py:153 | FL finished in 8.343552473999807
02/13/2024 15:56:02:INFO:FL finished in 8.343552473999807
INFO flwr 2024-02-13 15:56:02,442 | app.py:226 | app_fit: losses_distri

Server-side evaluation loss 0.06007593870162964 / accuracy 0.625


In [17]:
# Show thehistory of trining 
print(history)

History (loss, distributed):
	round 1: 0.048256014115535296
	round 2: 0.047671017165367424
	round 3: 0.04705386465558639
	round 4: 0.04648680182603689
	round 5: 0.04592530028178142
	round 6: 0.0454058895030847
	round 7: 0.04487835386624703
	round 8: 0.04436897629728684
	round 9: 0.04394516492119202
	round 10: 0.04349827952682972
History (loss, centralized):
	round 0: 0.06630590558052063
	round 1: 0.06550507992506027
	round 2: 0.06483973562717438
	round 3: 0.06413096934556961
	round 4: 0.06348368525505066
	round 5: 0.06283245980739594
	round 6: 0.06223991513252258
	round 7: 0.061645135283470154
	round 8: 0.06106271594762802
	round 9: 0.06058136373758316
	round 10: 0.06007593870162964
History (metrics, centralized):
{'accuracy': [(0, 0.625), (1, 0.625), (2, 0.625), (3, 0.625), (4, 0.625), (5, 0.625), (6, 0.625), (7, 0.625), (8, 0.625), (9, 0.625), (10, 0.625)]}


### 5. Store tests 

<div style="display:flex">
<div>

The initial phase of model testing involves utilizing test loaders on test nodes to evaluate the model's performance. This testing phase launches assessments on designated test nodes, generating results that are subsequently stored in a database. These stored results serve the purpose of enabling later comparisons and evaluations among different models

The tutorial labeled as `5_LearningResults-checkpoint` demonstrates diverse results and outputs accessible from the learning process. It elaborates on the multifaceted advantages and applications derived from these outcomes, emphasizing how each result can be effectively utilized for various purposes within the context of learning and model evaluation.
</div>
<div>

<img src='../Images/modeltest.png' width="300" style="display: block  ; margin : 0 auto" />
</div>

</div>


In [18]:
# Make some plots 
global_accuracy,global_loss = ppl_1.server.accuracies,ppl_1.server.losses
results_dict = {
    ('LR: 0.001, Optimizer: SGD', 'accuracy'): global_accuracy,
    ('LR: 0.001, Optimizer: SGD', 'loss'): global_loss
}
plotter = AccuracyLossPlotter(results_dict)

plotter.plot_accuracy_loss()

(DefaultActor pid=29969) epsilon of client 1 : eps = 0
(DefaultActor pid=29969) [Client 0] evaluate, config: {}


In [19]:
# Test the Modal on the Test nodes and store the results on the DB
ppl_1.auto_test()

[{'node_name': 'Midwest',
  'classification_report': "{'confusion matrix': {'TP': 0, 'FP': 4, 'FN': 0, 'TN': 14}, 'Accuracy': 0.778, 'Sensitivity/Recall': 0.0, 'Specificity': 0.778, 'PPV/Precision': 0.0, 'NPV': 1.0, 'F1-score': 0.0, 'False positive rate': 0.222, 'True positive rate': 0.0}"},
 {'node_name': 'South',
  'classification_report': "{'confusion matrix': {'TP': 2, 'FP': 5, 'FN': 2, 'TN': 23}, 'Accuracy': 0.781, 'Sensitivity/Recall': 0.5, 'Specificity': 0.821, 'PPV/Precision': 0.286, 'NPV': 0.92, 'F1-score': 0.364, 'False positive rate': 0.179, 'True positive rate': 0.5}"},
 {'node_name': 'West',
  'classification_report': "{'confusion matrix': {'TP': 11, 'FP': 11, 'FN': 10, 'TN': 61}, 'Accuracy': 0.774, 'Sensitivity/Recall': 0.524, 'Specificity': 0.847, 'PPV/Precision': 0.5, 'NPV': 0.859, 'F1-score': 0.512, 'False positive rate': 0.153, 'True positive rate': 0.524}"},
 {'node_name': 'Northeast',
  'classification_report': "{'confusion matrix': {'TP': 6, 'FP': 6, 'FN': 6, 'TN':